In [1]:
## import
import numpy as np
import pandas as pd
import re
# import re
from os.path import join

from tools import *

# from sklearn.decomposition import LatentDirichletAllocation
# from scipy.spatial import distance

# import matplotlib.pyplot as plt

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"
res_patient_subgroup_prefix = "/data/liu/adverse_events/patient_subgroup"
dis_sum_prefix = "/data/liu/mimic3/NOTESEVENT_EXTRACTION/ALL"
dis_sum_keysessions_prefix = "/data/liu/mimic3/NOTESEVENT_EXTRACTION/KEY_SESSIONS"
clamp_dis_sum_prefix = "/data/liu/mimic3/CLAMP_NER/input/discharge_summary"
clamp_dis_sum_keysessions_prefix = "/data/liu/mimic3/CLAMP_NER/input/keysessions"

In [2]:
## Original NOTESEVENTS
notes_df = read_data(join(read_prefix,"NOTEEVENTS"),
#                      usecols=['HADM_ID','DESCRIPTION','TEXT'],
                     dtype={'HADM_ID':str})
notes_df.head()

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [7]:
## check number of rows for NOTESEVENTS
print(len(notes_df[notes_df['CATEGORY']=='Discharge summary']))

print(len(notes_discharge_df[['TEXT']].dropna()))

59652

In [4]:
print("Original NOTESEVENTS: {:,d} episodes".format(len(notes_df.HADM_ID.unique())))
print("NOTESEVENTS belonging to Discharge Summary: {:,d} episodes".format(len(notes_df[notes_df['CATEGORY']=='Discharge summary'].HADM_ID.unique())))

Original NOTESEVENTS: 58,362 episodes
NOTESEVENTS belonging to Discharge Summary: 52,726 episodes


## Extract rows with "CATEGORY"
1. notes_df['CATEGORY']=='Discharge summary',  (patient asc, episode asc, discharge asc)
2. filter condition: ['TXEXT'] contain "Hospital Course:"

In [38]:
## only run for one time.  rank summary for each hadm_id
# cols = ['SUBJECT_ID', 'HADM_ID', 'TEXT']
# notes_discharge_df = notes_df[notes_df['CATEGORY']=='Discharge summary'][cols]
# notes_discharge_df['ROW_ID']=notes_discharge_df.index
# notes_discharge_df = notes_df[notes_df['CATEGORY']=='Discharge summary'][cols]

notes_discharge_df=notes_discharge_df.sort_values(by=['SUBJECT_ID','HADM_ID'])
notes_discharge_df['GROUP_RANK']=notes_discharge_df.groupby(['SUBJECT_ID','HADM_ID'])['ROW_ID'].rank(method='dense').astype(int)
notes_discharge_df.head()

In [4]:
## only run for one time: save discharge summary by folder(subject), each episode in one file

def save_keysessions_byfolder(notes_discharge_df,file="Discharge summary_%d",prefix=dis_sum_prefix, type="TXT"):
    for subject_id, group in notes_discharge_df.groupby(['SUBJECT_ID']):
        hadm_ids = group['HADM_ID'].unique()
        ## create folder ~/subject_id/hadm_id
        cols = ['GROUP_RANK','TEXT']
        
        for hadm_id, subgroup in group.groupby(['HADM_ID']):
            write_dir = join(prefix,str(subject_id),hadm_id)
            create_folder(write_dir)
#             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
#             dir = join(dis_sum_prefix,str(subject_id))    
            
            for _, row in subgroup[cols].iterrows():
                file_name = file%row['GROUP_RANK']
                write2txt(row['TEXT'],join(write_dir,file_name))

# def save_dissummary_byfolder(notes_discharge_df):
#     for subject_id, group in notes_discharge_df.groupby(['SUBJECT_ID']):
#         hadm_ids = group['HADM_ID'].unique()
#         ## create folder ~/subject_id/hadm_id
#         for hadm_id, subgroup in group.groupby(['HADM_ID']):
#             write_dir = join(dis_sum_prefix,str(subject_id),hadm_id)
#             create_folder(write_dir)
# #             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
# #             dir = join(dis_sum_prefix,str(subject_id))        
#             for _, row in subgroup[['GROUP_RANK','TEXT']].iterrows():
#                 file_name = "Discharge summary_%d" % row['GROUP_RANK']
#                 write2txt(row['TEXT'],join(write_dir,file_name))

def save_dissummary_byfile(notes_discharge_df):
    for subject_id, group in notes_discharge_df.groupby(['SUBJECT_ID']):
        hadm_ids = group['HADM_ID'].unique()
        ## create folder ~/subject_id/hadm_id
        for hadm_id, subgroup in group.groupby(['HADM_ID']):
#             write_dir = join(dis_sum_prefix,str(subject_id),hadm_id)
#             create_folder(write_dir)
#             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
#             dir = join(dis_sum_prefix,str(subject_id))        
            for _, row in subgroup[['GROUP_RANK','TEXT']].iterrows():
                file_name = "%s_%s_%d" % (str(subject_id),hadm_id,row['GROUP_RANK'])
                write2txt(row['TEXT'],join(clamp_dis_sum_prefix,file_name))
                
                
## SAVE discharge summary by folder, a file for a episode
save_dissummary_byfile(notes_discharge_df)
## SAVE discharge summary in to one file
# write2file(notes_discharge_df[['SUBJECT_ID', 'HADM_ID', 'GROUP_RANK', 'TEXT']],join(dis_sum_prefix,"Discharge summary_All"))

In [2]:
notes_discharge_df=read_data(join(dis_sum_prefix,"Discharge summary_All"),dtype={"HADM_ID":str})
print(len(notes_discharge_df))
notes_discharge_df.head()

59652


,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT
0,3,145834,1,Admission Date: [**2101-10-20**] Discharg...
1,4,185777,1,Admission Date: [**2191-3-16**] Discharge...
2,6,107064,1,Admission Date: [**2175-5-30**] Dischar...
3,9,150750,1,Admission Date: [**2149-11-9**] Dischar...
4,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi..."


In [12]:
print("Discharge Summaries containing Hospital Course: {:,d} episodes".format(len(notes_discharge_df.HADM_ID.unique())))

Discharge Summaries containing Hospital Course: 48,520 episodes


In [8]:
notes_discharge_df=notes_discharge_df[notes_discharge_df['TEXT'].str.contains('Hospital Course:', case=False)]
print(len(notes_discharge_df))
notes_discharge_df.head()

50219


,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT
2,6,107064,1,Admission Date: [**2175-5-30**] Dischar...
3,9,150750,1,Admission Date: [**2149-11-9**] Dischar...
5,10,184167,1,Admission Date: [**2103-6-28**] Dischar...
6,11,194540,1,Admission Date: [**2178-4-16**] ...
7,12,112213,1,Admission Date: [**2104-8-7**] Discharge ...


## Fix Bugs
I think there is an error in the code for extracting the hospital course sections. I looked at a few samples in /data/liu/mimic3/CLAMP_NER/input/hospital_course and their discharge summaries in /data/liu/mimic3/NOTESEVENT_EXTRACTION/ALL/ and also in /data/liu/mimic3/CLAMP_NER/input/discharge_summary. For example see these patients: 29542_196600_1.txt and 19583_189695_1.txt. The extracted hospital course files don't match the section in the discharge summary. Can you check your code? Also please share the code.


Check step:
1. from notes_df -> notes_discharge_df

In [1]:
## import
import numpy as np
import pandas as pd
import re
# import re
from os.path import join

from tools import *

# from sklearn.decomposition import LatentDirichletAllocation
# from scipy.spatial import distance

# import matplotlib.pyplot as plt

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"
res_patient_subgroup_prefix = "/data/liu/adverse_events/patient_subgroup"
dis_sum_prefix = "/data/liu/mimic3/NOTESEVENT_EXTRACTION/ALL"
dis_sum_keysessions_prefix = "/data/liu/mimic3/NOTESEVENT_EXTRACTION/KEY_SESSIONS"
clamp_dis_sum_prefix = "/data/liu/mimic3/CLAMP_NER/input/discharge_summary"
clamp_dis_sum_keysessions_prefix = "/data/liu/mimic3/CLAMP_NER/input/keysessions"

In [3]:
## Revised Section Extraction Script
def re_split(string, regex = r"\n\n[A-Z][A-Za-z/ ]+:"):
    """Split Discharge Summary to mulitple sections according to respective titles using regex rules

    Return:
    ------
    {titles:sections}: dict
        titiles : list. 
            Transfer all titles to lower case, making it easier to check and match
        sections: list. 
            len(sections)=len(titles)+1. The first section doesn't have title as it is not beginning 
            with "\n"       
    """
    return dict(zip(map(str.lower,re.findall(regex, string)), re.split(regex, string)[1:]))
    
def extract_sections(text,rule_index = 0, rerules = [
    r"\n[A-Z][A-Z/ ]+:",
    r"\n\n[A-Z][A-Z/ ]+:",
    r"\n\n[A-Z][A-Za-z/ ]+:",
    r"\n[A-Z][A-Za-z/ ]+:"
], selected_titles = ["hospital course","medical history","medications on admission"]):
    """
    
    Step:
    -----
    1) split original text to multiple sections with titles: dict{title -> section}
    2) select key sections by matching their titles to selected titles
    """
    section_dict = re_split(text, rerules[rule_index])
    ## selected title -> matched title
    titles_dict =dict([(selected_title, title) for title in section_dict.keys() 
                       for selected_title in selected_titles if selected_title in title])
    ## get actually extracted titles that are matched to selected titles
    section_titles = [titles_dict.get(title, None) for title in selected_titles]
    
    return tuple([(',').join(filter(None, section_titles))]+
                 [section_dict.get(title, None) for title in section_titles])
    
                    
#     for re_rule in rerules:
#         section_dict=re_split(text,re_rule)
#         keys = section_dict.keys
#         matched_key
        
    

#     indexes = [search_title_index(titles, section_title) for section_title in section_titles]
    
#     return tuple([('').join(titles)]+ [item_of_list(sections,index) for index in indexes])

In [27]:
from S2.dis_sum_extraction import *

Rule 0 :
rule only df row num 18287 
 episode num 15736 

remain df row num 41365 
 episode num 38515 


Rule 1 :
rule only df row num 37401 
 episode num 36505 

remain df row num 3964 
 episode num 3674 


Rule 2 :
rule only df row num 53 
 episode num 53 

remain df row num 3911 
 episode num 3625 




In [28]:
notes_discharge_df=read_data(join("/data/liu/mimic3/CLAMP_NER/input","ReRule0_Discharge summary_Remain"),dtype={"HADM_ID":str})
print(len(notes_discharge_df))
notes_discharge_df.head()

41365


,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,LINE NUMBER,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
0,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi...",NaN,"None,None,None",NaN,NaN,NaN
1,11,194540,1,Admission Date: [**2178-4-16**] ...,NaN,"None,None,None",NaN,NaN,NaN
2,13,143045,2,"Name: [**Known lastname 9900**], [**Known fir...",NaN,"None,None,None",NaN,NaN,NaN
3,17,161087,1,Admission Date: [**2135-5-9**] D...,NaN,"None,None,None",NaN,NaN,NaN
4,17,194023,1,Admission Date: [**2134-12-27**] ...,NaN,"None,None,None",NaN,NaN,NaN


In [29]:
notes_discharge_df = run_rule(notes_discharge_df,1)

In [5]:
is_NaN = notes_discharge_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = notes_discharge_df[row_has_NaN]
print(len(rows_with_NaN))
rows_with_NaN.head()

41365


,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,LINE NUMBER,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
0,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi...",NaN,"None,None,None",NaN,NaN,NaN
1,11,194540,1,Admission Date: [**2178-4-16**] ...,NaN,"None,None,None",NaN,NaN,NaN
2,13,143045,2,"Name: [**Known lastname 9900**], [**Known fir...",NaN,"None,None,None",NaN,NaN,NaN
3,17,161087,1,Admission Date: [**2135-5-9**] D...,NaN,"None,None,None",NaN,NaN,NaN
4,17,194023,1,Admission Date: [**2134-12-27**] ...,NaN,"None,None,None",NaN,NaN,NaN


In [23]:
rule_only_df, remain_df = [x for _, x in notes_discharge_df.groupby(notes_discharge_df['TITLE'].isna())]
print("rule only df row num %d \n episode num %d \n\nremain df row num %d \n episode num %d \n\n"%(
    len(rule_only_df),len(rule_only_df['HADM_ID'].unique()),len(remain_df),len(remain_df['HADM_ID'].unique())))

rule only df row num 41365 
 episode num 38515 

remain df row num 18287 
 episode num 15736 




In [29]:
rule_only_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,LINE NUMBER,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
4,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi...",NaN,"None,None,None",NaN,NaN,NaN
6,11,194540,1,Admission Date: [**2178-4-16**] ...,NaN,"None,None,None",NaN,NaN,NaN
9,13,143045,2,"Name: [**Known lastname 9900**], [**Known fir...",NaN,"None,None,None",NaN,NaN,NaN
10,17,161087,1,Admission Date: [**2135-5-9**] D...,NaN,"None,None,None",NaN,NaN,NaN
11,17,194023,1,Admission Date: [**2134-12-27**] ...,NaN,"None,None,None",NaN,NaN,NaN


In [2]:
# notes_discharge_df=read_data(join(dis_sum_prefix,"Discharge summary_All"),dtype={"HADM_ID":str})
# len(notes_discharge_df)
# # add_cols = ["TITLE", "LINE NUMBER", "HOSPITAL COURSE", "MEDICAL HISTORY", "MEDICATIONS ON ADMISSION"]
# # notes_discharge_df[add_cols] = pd.DataFrame(notes_discharge_df['TEXT'].apply(lambda x: extract_sections(x,0)).tolist())
notes_discharge_df=read_data(join("/data/liu/mimic3/CLAMP_NER/input","ReRule0_Discharge summary_All"),dtype={"HADM_ID":str})
notes_discharge_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,LINE NUMBER,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
0,3,145834,1,Admission Date: [**2101-10-20**] Discharg...,"hospital course by system:,past medical histor...","134,28,309",\n\n1. CARDIOVASCULAR: The patient was initi...,\n 1. Coronary artery disease with diffuse 3-...,\n 1. Amiodarone 400 mg p.o. b.i.d. (through ...
1,4,185777,1,Admission Date: [**2191-3-16**] Discharge...,brief summary of hospital course by issue/syst...,"108,34,203",\n\n1. PULMONARY ISSUES: From the Emergency ...,\n1. Human immunodeficiency virus diagnosed i...,\n 1. Prednisone 40 mg once per day times thr...
2,6,107064,1,Admission Date: [**2175-5-30**] Dischar...,"hospital course:,past medical history:,medicat...","40,21,28","On [**2175-5-30**], the patient went to the\...","End stage renal disease, secondary to\nmalig...",Unknown.\n
3,9,150750,1,Admission Date: [**2149-11-9**] Dischar...,"hospital course:,past medical history:,admissi...","75,25,27",1. NEUROLOGIC: Left putaminal bleed\nsecon...,Hypertension.\n,The patient was on no medications\nupon admi...
4,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi...",NaN,NaN,NaN,NaN,NaN


In [10]:
## check rows with null value
is_NaN = notes_discharge_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = notes_discharge_df[row_has_NaN]
rows_with_NaN.head()
# len(notes_discharge_df[notes_discharge_df['TITLE'].notna()]['HADM_ID'].unique())

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,LINE NUMBER,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
4,9,150750,2,"Name: [**Known lastname 10050**], [**Known fi...",NaN,NaN,NaN,NaN,NaN
5,10,184167,1,Admission Date: [**2103-6-28**] Dischar...,"hospital course:,medications:","35,None,81",1.) Respiratory: Baby Girl [**Known lastnam...,NaN,None.\n
6,11,194540,1,Admission Date: [**2178-4-16**] ...,NaN,"None,None,None",NaN,NaN,NaN
8,13,143045,1,Admission Date: [**2167-1-8**] Discharg...,"prior medical history:,discharge medications:","None,33,140",NaN,"She has diabetes mellitus, type 2,\ndiagnose...",\n1. Metoprolol 100 mg t.i.d.\n2. Aspirin 32...
9,13,143045,2,"Name: [**Known lastname 9900**], [**Known fir...",NaN,NaN,NaN,NaN,NaN


In [11]:
rows_with_NaN["MEDICATIONS ON ADMISSION"].unique()

array([nan, '  None.\n',
       '\n1.  Metoprolol 100 mg t.i.d.\n2.  Aspirin 325 mg p.o. q.d.\n3.  Captopril 50 mg p.o. t.i.d.\n4.  Lipitor 20 mg p.o. q.d.\n5.  Lasix 20 mg p.o. b.i.d. for one week.\n6.  Potassium chloride 10 mEq p.o. b.i.d. for one week.\n7.  Metformin 500 mg p.o. b.i.d.;  follow your blood sugar\nlevels and follow up with [**Doctor First Name 8392**].\n8.  Percocet one to two tablets every four hours as needed\nfor pain.\n9.  Colace 100 mg p.o. b.i.d. as needed for constipation.\n\n[**Last Name (STitle) 54220**]owup appointments are with Dr. [**Last Name (STitle) **], her primary\ncare physician, [**Name10 (NameIs) **] one to two weeks, Dr. [**Last Name (STitle) 911**], her\ncardiologist, in two to three weeks, Dr. [**First Name (STitle) **] from [**Doctor First Name 8392**]\nas directed, and Dr. [**Last Name (Prefixes) **] in one month. She is also\nrecommended to follow up in [**Hospital1 18**] Wound Care Clinic in one to\ntwo weeks to remove her staples.\n\n\n    

In [28]:
reversed("ADMISSION MEDICATIONS") in "MEDICATIONS ON ADMISSIONS"

TypeError: 'in <string>' requires string as left operand, not reversed

## deprecated: Extract sessions
BRIEF SUMMARY OF HOSPITAL COURSE, PAST MEDICAL HISTORY and MEDICATIONS ON ADMISSION

In [58]:
## split by regex rules
def re_split(string, regex = r"\n\n[A-Z][A-Za-z/ ]+:",length=False):
    if length: return len(re.findall(regex, string))
    else: 
        return re.findall(regex, string), re.split(regex, string)

    
## Titles of Wanted sections
## BRIEF SUMMARY OF HOSPITAL COURSE: hospital course, PAST MEDICAL HISTORY and MEDICATIONS ON ADMISSION
# section_titles = ["HOSPITAL COURSE", "MEDICAL HISTORY", "MEDICATIONS ON ADMISSION"]


def search_title_index(titles, substring):
    for i in range(len(titles)):
        if substring.lower() in titles[i].lower():
            return i
    return None

def item_of_list(list, index):
    if index: return list[index]
    else: return None

def extract_sections(text,rerules = [
    r"\n[A-Z][A-Z/ ]+:",
    r"\n\n[A-Z][A-Z/ ]+:",
    r"\n\n[A-Z][A-Za-z/ ]+:",
    r"\n[A-Z][A-Za-z/ ]+:"
], section_titles = ["HOSPITAL COURSE", "MEDICAL HISTORY", "MEDICATIONS ON ADMISSION"]):
    # option 1
    titles, sections=re_split(text, rerules[0])
    
    if(len(titles)>20): 
        titles, sections =re_split(text, rerules[1])
        
    if(len(titles)<5): 
        titles, sections=re_split(text, rerules[2])
    if(len(titles)<2): 
        titles, sections=re_split(text, rerules[3])
    print(titles)
    indexes = [search_title_index(titles, section_title) for section_title in section_titles]
    
    return tuple([('').join(titles)]+ [item_of_list(sections,index) for index in indexes])
    ## option 2: bad,  count -> {<5: 20608; >20: 2328 }
#     split_lists=[re_split(text, rerule) for rerule in rerules]
#     split_lens=[len(split) for split in split_lists]
#     best_split_index = np.argsort(split_lens)[len(split_lens)//2]
        
#     return split_lens[best_split_index], ('').join(split_lists[best_split_index])   


# notes_discharge_df['SPLIT_N'], notes_discharge_df['TITLE'] = zip(*(notes_discharge_df['TEXT']
#                                                                    .apply(extract_sections).tolist()))

In [7]:
# res = notes_discharge_df['TEXT'].apply(extract_sessions).tolist()
# pd.DataFrame(res)
notes_discharge_df[["TITLE", "HOSPITAL COURSE", 
                    "MEDICAL HISTORY", "MEDICATIONS ON ADMISSION"]] = pd.DataFrame(notes_discharge_df['TEXT']
                                                                                   .apply(extract_sessions).tolist())

In [43]:
## EG. check SPLIT_N too high or low for all records
notes_discharge_df_low = notes_discharge_df[notes_discharge_df['SPLIT_N']<5]
notes_discharge_df_high = notes_discharge_df[notes_discharge_df['SPLIT_N']>20]
print("<5: %d" % len(notes_discharge_df_low))
print(">20: %d" % len(notes_discharge_df_high))


def show_one_note(df,id,count=None):
    if not count:
        index=df.index
        one_note=df['TEXT'][index[0]]
        return one_note
    else:
        return df[df['SPLIT_N']==count]

# one_note = show_one_note(notes_discharge_df_high,1,67)['TEXT'][40594]
one_note=show_one_note(notes_discharge_df_high,0)
print(one_note)

## single text example
split_rule = [
    r"\n[A-Z/ ]+:",
    r"\n[A-Z][A-Z/ ]+:",
    r"\n\n[A-Z/ ]+:",
    r"\n\n[A-Za-z/ ]+:",
    r"\n\b[A-Za-z/ ]{2,}\b:",
    r"\n\n\b[A-Za-z/ ]{2,}\b:",
    r"\n\n[A-Z]+[A-Za-z/ ]+:",
]

# text_test = "Admission Date:  [**2149-11-9**]       Discharge Date: [**2149-11-13**]\n\nDate of Birth:   [**2108-1-26**]       Sex:  M\n\nService:  NEUROLOGY\n\nCHIEF COMPLAINT:  Weakness, inability to talk.\n\nHISTORY OF THE PRESENT ILLNESS:  This is a 41-year-old\nAfrican-American male with a history of hypertension who was\nin his usual state of health until about 10:25 a.m. on the\nmorning of admission.  He had gone to use the restroom and a\nfew minutes later his family found him slumped onto the\nfloor, apparently unable to talk and with weakness in his\nright arm and leg.  EMS was called and he was brought into\nthe Emergency Department at [**Hospital1 18**].\n\nThe patient has not had strokes or previous similar symptoms.\nHe has a history of hypertension but no history of cardiac\nsymptoms.  The patient was unable to talk for examination and\nno family members were present at the bedside and were not at\nhome (apparently they were on the way to the Emergency Room).\nThe history was obtained by EMS.\n\nPAST MEDICAL HISTORY:  Hypertension.\n\nADMISSION MEDICATIONS:  The patient was on no medications\nupon admission.\n\nALLERGIES:  The patient has no known drug allergies.\n\nFAMILY HISTORY:  Unknown.\n\nSOCIAL HISTORY:  The patient lives [**Location (un) 6409**].\n\nPHYSICAL EXAMINATION ON ADMISSION:  During the five minute\nexamination, the patient became progressively less responsive\nand then vomited requiring intubation and paralytics during\nthe examination.  Vital signs:  Blood pressure 229/137, heart\nrate 85, respiratory rate 20, temperature 98.8.  General:\nThis was a well-developed African-American male.  HEENT:  The\nneck was supple without lymphadenopathy or thyromegaly.\nCardiovascular:  Regular rate and rhythm.  No murmurs, rubs,\nor gallops noted.  Lungs:  Clear to auscultation bilaterally.\nAbdomen:  Soft, nontender, nondistended with no hepatomegaly.\nExtremities:  No clubbing, cyanosis or edema noted.\nNeurologic:  On the mental status examination, the patient\nwas alert but nonverbal.  Language was aphasic but can\ncomprehend one-step commands.  There was no evidence of\nneglect.  The patient can follow one-step commands.  On the\ncranial nerve examination, his optic disk margins were sharp.\nHis extraocular eye movements were intact.  His pupils were\nequal and reactive to light.  His face was symmetrical at\nrest.  Motor:  He had normal bulk and tone with no tremors.\nHis power was 0/5 in the right arm and leg.  His left arm and\nleg was apparently [**5-5**].  Sensory:  He was reactive to noxious\nstimuli in all four extremities.  His reflexes were minimal\nthroughout.  His toes were mute bilaterally.  Coordination\nand gait examination was not tested.\n\nLABORATORY/RADIOLOGIC DATA:  White count 7.5, hematocrit 45,\nplatelets 258,000.  PT 12.7, PTT 21.7, INR 1.1.\n\nThe EKG was normal sinus rhythm at 99 with normal axis and\nintervals.  LVH was noted.  There were T wave inversions in\nI, II, III, aVL, V1 through V3.\n\nSerum and urine toxicology was negative.  ESR was 18.  The\nurinalysis was negative.  CT scan of the head showed 55 by 55\nby 21 mm left putaminal bleed with a 5 mm midline shift to\nthe right.\n\nChest x-ray showed cardiomegaly with CHF.\n\nHOSPITAL COURSE:  1.  NEUROLOGIC:  Left putaminal bleed\nsecondary to presumed hypertension.  The patient's blood\npressure was initially controlled with a Nipride drip to keep\nthe cephalic pressure between 130-160.  The Nipride drip was\nthen weaned as labetalol IV was added along with a\nnicardipine drip.  The patient was also given Dilantin load\nfollowed by a maintenance dose of 400 mg q.d.  His level has\nbeen therapeutic around 20.  Considerations for a CTA and\nangiogram for possible AVM or aneurysm were considered but\nthe patient's renal function and health status did not allow\nsuch studies to be done at the current time.\n\nRepeated noncontrast CT of the head revealed no increase in\nthe bleeding but showed subsequently increases in cerebral\nedema with some pressure onto the brain stem.  The patient's\nneurological examination did deteriorate to the point that he\nonly had reactive pupils that were equal and intact corneal\nreflexes.  In addition, he had a gag reflex.  However, he\nbegan to not move any extremities and not respond to any\nnoxious stimuli in the four extremities.\n\nBy 4:00 p.m. on [**2149-11-13**], his neurological condition\ndeteriorated to the point that his pupils were dilated and\nnonreactive.  He lost gag and corneal reflex.  Calorics were\ndone at this time which revealed no response.  Neurosurgery\nwas reconsulted for a question of craniotomy but they felt\nthat the patient would not be a candidate for the surgery in\nthat it would not help him.  Mannitol was then started along\nwith hyperventilation to keep the PC02 below 30 in an attempt\nto decrease the cerebral edema.\n\nDespite clinical examination that the patient is clinically\nbrain dead, the family wishes to continue aggressive\ntreatment.  The family requested that the patient be\ntransferred to the [**Hospital6 1129**].  However,\nDr. [**Last Name (STitle) 54029**] at the [**Hospital3 **] Stroke Service said\nthat the patient would not be accepted at the [**Hospital3 **] given that there would be no other intervention that\nwould be done differently at [**Hospital3 **].\n\n2.  CARDIOVASCULAR:  As mentioned above, the patient was\ninitially started on a Nipride drip to control the blood\npressure between 130-160.  However, he was slowly weaned off\nthe Nipride to be switched over to Cardene drip.  The Cardene\ndrip was then weaned to allow for the addition of p.o.\nLabetalol.  Three serial CKs and troponins were checked and\nfound to be negative, thus, ruling the patient out for a\nmyocardial infarction.\n\n3.  PULMONARY:  The patient was initially intubated when\nfirst seen down in the Emergency Room for airway protection\nsince he vomited.  He then showed a left lower lobe pneumonia\non the chest x-ray.  He was started on levofloxacin and\nFlagyl since the day of admission for aspiration pneumonia.\nVancomycin was added on day number four of hospitalization\ndue to sputum cultures showing Staphylococcus aureus\ncoagulase-positive organisms and gram-negative rods.\nSusceptibilities are still pending.  The patient did have one\nor two episodes of desaturation secondary to breathing trials\nand increased secretion in his nostrils.  In an attempt to\ndecongest him, saline nasal spray was started.\n\n4.  RENAL:  Acute renal failure secondary to dehydration.\nThe patient's creatinine upon admission was 1.4 and continued\nto rise to 2 as he started third spacing his fluids.  Urine\nelectrolytes were checked which showed a phena of less than 1\nwhich is evidence of the patient being dehydrated and\nprerenal.  He was subsequently given IV fluids and his\ncreatinine improved to 1.6.  A renal ultrasound was done\nwhich showed no evidence of hydronephrosis.\n\n5.  INFECTIOUS DISEASE:  Leukocytosis:  Given the aspiration\npneumonia and elevated white count of 18.6, levofloxacin and\nFlagyl were started on the day of admission.  Blood cultures\nwere obtained.  The first two sets showed one out of four\nblood cultures that grew staphylococcus coagulase-negative.\nSince this was deemed a contaminent, no further antibiotics\nwere given.  However, on day number four of hospitalization,\nhis sputum culture returned showing Staphylococcus\ncoagulase-positive organisms so vancomycin was added.  More\nblood cultures were drawn but nothing has grown to date.\n\n6.  GASTROINTESTINAL:  The patient was given and orogastric\ntube for tube feeds.  He had some high residuals so the tube\nfeeds were stopped.  Reglan was then administered to promote\nGI motility so that the tube feeds could be restarted.\n\n7. FLUIDS, ELECTROLYTES, AND NUTRITION:  Hyponatremia:  The\npatient's sodium slowly fell from 136 upon admission to 130\non the fourth day of hospitalization.  It was felt that this\nwas due to salt wasting from his cerebral problems so he was\nthen fluid restricted.  Although he has acute renal failure\nsecondary to dehydration and third spacing, it was felt that\nhe needed to be fluid restricted given that the overall\nhyponatremia was more likely due to salt wasting.\n\n\n\n\n                            [**First Name8 (NamePattern2) **] [**First Name8 (NamePattern2) 5930**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 6125**]\n\nDictated By:[**Last Name (NamePattern1) 4270**]\nMEDQUIST36\n\nD:  [**2149-11-13**]  07:03\nT:  [**2149-11-13**]  19:46\nJOB#:  [**Job Number 54030**]\n"
text_test="Admission Date:  [**2134-12-27**]              Discharge Date:   [**2134-12-31**]\n\nDate of Birth:  [**2087-7-14**]             Sex:   F\n\nService: CARDIOTHORACIC\n\nAllergies:\nBactrim / Ampicillin / Remeron\n\nAttending:[**First Name3 (LF) 1283**]\nChief Complaint:\nHistory of stroke\n\nMajor Surgical or Invasive Procedure:\n[**2134-12-27**] Minimally invasive closure of patent foramen ovale\n\n\nHistory of Present Illness:\nMrs. [**Known lastname 11679**] is a 47 year old female who suffered a cerebellar\nstroke in [**2134-3-9**]. Workup at that time revealed a patent\nforamen ovale. She is currently followed by Dr.\n[**Last Name (STitle) 1693**](neurologist) from the [**Hospital1 18**]. Full hypercoagulability\nworkup was unremarkable. Since [**Month (only) 956**], she has had no other\nneurological events. In preperation for surgical intervention,\nshe underwent cardiac catheterization in [**Month (only) **] which showed\nnormal coronary arteries and normal left ventricular function.\n\nPast Medical History:\nPatent foramen ovale; History of Stroke/TIA; Depression;\nAnxiety; Borderline Hyperlipidemia; Herniation of Cervical\nDiscs; Patella-Femoral Syndrome; s/p Bunionectomies\n\nSocial History:\nDenies tobacco. Admits to occasional ETOH. She is an employee of\nthe [**Hospital1 18**] in the Neuro-Pysch Department. She is married with two\nchildren. She denies IVDA and recreational drugs.\n\nFamily History:\nFather underwent CABG at age 72. Cousin died of an MI at age 46.\n\nPhysical Exam:\nVitals: BP 114/68, HR 90, RR 14\nGeneral: well developed female in no acute distress\nHEENT: oropharynx benign,\nNeck: supple, no JVD, no carotid bruits\nHeart: regular rate, normal s1s2, no murmur or rub\nLungs: clear bilaterally\nAbdomen: soft, nontender, normoactive bowel sounds\nExt: warm, no edema, no varicosities\nPulses: 2+ distally\nNeuro: nonfocal\n\n\nPertinent Results:\n[**2134-12-31**] 06:15AM BLOOD WBC-6.6# RBC-2.98* Hgb-9.1* Hct-26.1*\nMCV-88 MCH-30.6 MCHC-35.0 RDW-13.1 Plt Ct-192\n[**2134-12-27**] 06:19PM BLOOD WBC-10.5 RBC-3.42*# Hgb-10.5*# Hct-30.0*\nMCV-88 MCH-30.8 MCHC-35.2* RDW-12.6 Plt Ct-138*\n[**2134-12-31**] 06:15AM BLOOD Glucose-121* UreaN-12 Creat-0.7 Na-140\nK-5.1 Cl-106 HCO3-28 AnGap-11\n[**2134-12-27**] 07:21PM BLOOD UreaN-11 Creat-0.8 Cl-112* HCO3-23\n[**2134-12-31**] 06:15AM BLOOD Calcium-8.1* Phos-3.2 Mg-2.0\n\nBrief Hospital Course:\nMrs. [**Known lastname 11679**] was admitted and underwent surgical closure of her\npatent foramen ovale. The operation was performed minimally\ninvasive and there were no complications. Following the\nprocedure, she was brought to the CSRU. She initially remained\nhypotensive, requiring volume and Neosynephrine. Within 24\nhours, she awoke neurologically intact and was extubated without\ndifficulty. By postoperative day two, she successfully weaned\nfrom inotropic support. She maintained stable hemodynamics and\ntransferred to the floor. On telemetry, she remained mostly in a\nnormal sinus rhythm with brief periods of accelerated junctional\nrhythm. She otherwise continued to make clinical improvements\nand was cleared for discharge on postoperative day four. She\nremained just on Aspirin therapy. Aggrenox was not resumed as\nher PFO was surgically repaired. At discharge, her systolic\nblood pressures were in the 100's with heart rate of 80-90. Her\nroom air saturations were 93% and she was ambulating without\ndifficulty. She had good pain control with Dilaudid and all\nwounds were clean, dry and intact.\n\nMedications on Admission:\nBupropion 150 [**Hospital1 **], Aggrenox qd, Centrum, Calcium, Erythromycin\neye gtts\n\nDischarge Medications:\n1. Bupropion 150 mg Tablet Sustained Release Sig: One (1) Tablet\nSustained Release PO BID (2 times a day).\nDisp:*60 Tablet Sustained Release(s)* Refills:*2*\n2. Docusate Sodium 100 mg Capsule Sig: One (1) Capsule PO BID (2\ntimes a day).\nDisp:*60 Capsule(s)* Refills:*2*\n3. Aspirin 81 mg Tablet, Delayed Release (E.C.) Sig: One (1)\nTablet, Delayed Release (E.C.) PO DAILY (Daily).\nDisp:*30 Tablet, Delayed Release (E.C.)(s)* Refills:*2*\n4. Hydromorphone 2 mg Tablet Sig: 1-2 Tablets PO every 6-8 hours\nas needed.\nDisp:*50 Tablet(s)* Refills:*0*\n\n\nDischarge Disposition:\nHome With Service\n\nFacility:\n[**Hospital 119**] Homecare\n\nDischarge Diagnosis:\nPatent foramen ovale - s/p surgical closure; History of\nStroke/TIA; Depression; Anxiety; Borderline Hyperlipidemia;\nHerniation of Cervical Discs; Patella-Femoral Syndrome; s/p\nBunionectomies\n\n\nDischarge Condition:\nGood\n\n\nDischarge Instructions:\nPatient may shower, no baths. No creams, lotions or ointments to\nincisions. No driving for at least one month. No lifting more\nthan 10 lbs for at least 10 weeks from the date of surgery.\nMonitor wounds for signs of infection. Please call with any\nconcerns or questions.\n\n\nFollowup Instructions:\nCardiac surgeon, Dr. [**Last Name (STitle) 1290**] in [**5-11**] weeks - call for appt,\n[**Telephone/Fax (1) 170**]. Local PCP, [**Last Name (NamePattern4) **]. [**Last Name (STitle) **] in [**3-11**] weeks - call for\nappt. Local cardiologist, Dr. [**Last Name (STitle) 11255**] in [**3-11**] weeks - call for\nappt\n\n\n\nCompleted by:[**2134-12-31**]"

In [66]:
notes_discharge_df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'GROUP_RANK', 'TEXT', 'TITLE',
       'HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION'],
      dtype='object')

In [83]:
## Only Run for One Time: save sessions for all episodes in one file
write2file(notes_discharge_df[['SUBJECT_ID', 'HADM_ID', 
                               'GROUP_RANK','TITLE',
                               'GROUP_RANK','HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION']],
          join(dis_sum_keysessions_prefix,"Key Sessions_All"))

In [5]:
def save_keysessions_byfolder(notes_discharge_df,file="Key sessions_%d",prefix=dis_sum_keysessions_prefix):
    for subject_id, group in notes_discharge_df.groupby(['SUBJECT_ID']):
        hadm_ids = group['HADM_ID'].unique()
        ## create folder ~/subject_id/hadm_id
        cols = ['GROUP_RANK','HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION']
        
        for hadm_id, subgroup in group.groupby(['HADM_ID']):
            write_dir = join(prefix,str(subject_id),hadm_id)
            create_folder(write_dir)
#             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
#             dir = join(dis_sum_prefix,str(subject_id))    
            
            for _, row in subgroup[cols].iterrows():
                file_name = file%row['GROUP_RANK']
                write2json(row.to_json(),join(write_dir,file_name))

In [3]:
def save_keysessions_byfile(notes_discharge_df,file="%s_%s_%d",prefix="/data/liu/mimic3/CLAMP_NER/input"):
    
    pastdrug_pre = join(prefix,"medical_history")
    course_pre = join(prefix,"hospital_course")
    admi_medi_pre = join(prefix,"medications_on_admission")
    
    for subject_id, group in notes_discharge_df.groupby(['SUBJECT_ID']):
        hadm_ids = group['HADM_ID'].unique()
        ## create folder ~/subject_id/hadm_id
        cols = ['GROUP_RANK','HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION']
        
        for hadm_id, subgroup in group.groupby(['HADM_ID']):
#             write_dir = join(prefix,str(subject_id),hadm_id)
#             create_folder(write_dir)
#             [create_folder(join(dis_sum_prefix,str(subject_id),hadm_id)) for hadm_id in hadm_ids]
#             dir = join(dis_sum_prefix,str(subject_id))    
            
            for _, row in subgroup[cols].iterrows():
                file_name = file%(str(subject_id), hadm_id, row['GROUP_RANK'])
                write2txt(row["HOSPITAL COURSE"],join(course_pre,file_name))
                write2txt(row["MEDICAL HISTORY"],join(pastdrug_pre,file_name))
                write2txt(row["MEDICATIONS ON ADMISSION"],join(admi_medi_pre,file_name))

#                 file_name = file%(str(subject_id), hadm_id, row['GROUP_RANK'])
# #                 print(('\n\n').join(row[cols[1:]].tolist()))
#                 write2txt(('\n\n').join(row[cols[1:]].tolist()),join(prefix,file_name))

In [17]:
notes_discharge_df = read_data(join(dis_sum_keysessions_prefix,"Key Sessions_All"))

In [7]:
### dropna: remove those discharge summaries having null value in any session of three
save_keysessions_byfile(notes_discharge_df.dropna())

In [15]:
for folder in ['HOSPITAL COURSE', 'MEDICAL HISTORY', 'MEDICATIONS ON ADMISSION']:    
    print("{:}: {:,d} episodes".format(folder,len(notes_discharge_df.dropna(subset=[folder]).HADM_ID.unique())))

HOSPITAL COURSE: 21,912 episodes
MEDICAL HISTORY: 19,601 episodes
MEDICATIONS ON ADMISSION: 15,520 episodes


In [23]:
print("{:}: {:,d} episodes".format("Discharge Summaries without null value in any Session",
                                  len(notes_discharge_df.dropna().HADM_ID.unique())))

Discharge Summaries without null value in any Session: 15,289 episodes


In [6]:
len(notes_discharge_df.SUBJECT_ID.unique())

37560

In [12]:
notes_discharge_df_withcourse = notes_discharge_df.dropna(subset=['HOSPITAL COURSE'])

In [15]:
len(notes_discharge_df_withcourse.SUBJECT_ID.unique())

17974

In [8]:
# save_keysessions_byfolder(notes_discharge_df)

## Check Titles of Sessions

In [12]:
notes_discharge_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,SPLIT_N,TITLE
2,6,107064,1,Admission Date: [**2175-5-30**] Dischar...,9,\nADMISSION DIAGNOSIS:\nHISTORY OF PRESENT ILL...
3,9,150750,1,Admission Date: [**2149-11-9**] Dischar...,10,\nCHIEF COMPLAINT:\nHISTORY OF THE PRESENT ILL...
5,10,184167,1,Admission Date: [**2103-6-28**] Dischar...,9,\nHISTORY OF PRESENT ILLNESS:\nPHYSICAL EXAMIN...
6,11,194540,1,Admission Date: [**2178-4-16**] ...,15,\n\nPHYSICAL EXAM:\n\nHISTORY:\n\nTECHNIQUE:\n...
7,12,112213,1,Admission Date: [**2104-8-7**] Discharge ...,9,\nHISTORY OF PRESENT ILLNESS:\nPAST MEDICAL HI...


In [5]:
titles = notes_discharge_df['TITLE'][2].split('\n')

In [6]:
print(notes_discharge_df['TEXT'][2])

Admission Date: [**2175-5-30**]        Discharge Date:  [**2175-6-15**]

Date of Birth:                    Sex:  F

Service:


ADMISSION DIAGNOSIS:  End stage renal disease, admitted for
transplant surgery.

HISTORY OF PRESENT ILLNESS:  The patient is a 65 year-old
woman with end stage renal disease, secondary to malignant
hypertension. She was started on dialysis in [**2174-2-7**].  She currently was on peritoneal dialysis and appears
to be doing well.  She has a history of gastric angiectasia
which she requires endoscopy. She was admitted on [**2175-5-30**] for
a scheduled living donor kidney transplant by her son, who is
the donor.  She does have a donor specific antibody (B-51)
and will have a final T & B cell class match prior to
transplantation.

PAST MEDICAL HISTORY:  End stage renal disease, secondary to
malignant hypertension on dialysis.  History of anemia
following gastric angiectasia.  She has no known history for
coronary artery disease for diabetes.

ALLERGIES:  No known 